IMPORTANT: Previously, you need to create a Twitter developer account [here](https://developer.twitter.com/en).

In [ ]:
!pip install tweepy

In [ ]:
import time
from datetime import datetime

import tweepy

import pymongo

In [ ]:
DATE_FORMAT = '%a %b %d %H:%M:%S +0000 %Y'

In [ ]:
# MongoDB connection data
HOST = 'localhost'
PORT = '27017'
DB_NAME = 'trending'

In [ ]:
# Twitter credentials
API_KEY = ''
API_SECRET_KEY = ''

ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

### Connecting to database

In [ ]:
db = None
try:
    db_client = pymongo.MongoClient(HOST + ':' + PORT)
    db = db_client[DB_NAME]
    print('Conecction established successfully!')
except pymongo.errors.ConnectionFailure:
    print('Connection cannot be established')

In [ ]:
USERS_TIMELINE_COL = db['users_timeline']
HISTORY_COL = db['history']

### Connecting to Twitter

In [ ]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)
#stream = tweepy.Stream(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

### Getting latest timeline for an user

In [ ]:
# https://docs.tweepy.org/en/latest/api.html#tweepy.API.user_timeline
tweets = api.user_timeline(screen_name = 'IvanDuque')

In [ ]:
# TODO: Modify previous code to get timeline older than 20 latest tweets
# Hint: Use the parameter since_id

In [ ]:
for tweet in tweets:
    tweet_dict = tweet._json
    tweet_dict['created_at'] = datetime.strptime(tweet_dict['created_at'], DATE_FORMAT)
    USERS_TIMELINE_COL.insert_one(tweet_dict)

### Getting history for a topic

In [ ]:
id = None
while True:
    try:
        # https://docs.tweepy.org/en/latest/api.html#tweepy.API.search_tweets
        tweets = api.search_tweets(q = 'Medellín', lang = 'es', tweet_mode = 'extended', max_id = id)
        for tweet in tweets:
            tweet_dict = tweet._json
            tweet_dict['created_at'] = datetime.strptime(tweet_dict['created_at'], DATE_FORMAT)
            HISTORY_COL.insert_one(tweet_dict)
        id = tweet.id
    except tweepy.TooManyRequests:
        print('Too many requests. Sleeping...')
        time.sleep(600)
        print('Getting back to action!')
        pass